#   Riccati equations


In this live script the Riccati equations in the context of the finite horizon 
linear quadratic control framework are implemented and the horizon is increased 
so that the solution to the algebraic Riccati equation is obtained.

Let us start by recalling the finite-horizon linear quadratic regulator framework. 
It is specified by a finite-horizon quadratic cost function

$$\sum_{k=0}^{h-1}x_k^{T}\,\,Qx_k+u_k^TRu_k+x_h^T Q_hx_h $$

and a linear model

$$ x_{k+1}=Ax_k+Bu_k$$

where $k\in\{0,\dots,h-1\}$. 

The optimal policy is 

$$u_k=K_kx_k,$$

where

$$K_k = -(R+B^T P_{k+1}B)^{-1}B^TP_{k+1}A,$$

and the $P_k$s are obtained by iterating, for $k\in \{h-1,h-2,\dots,0\}$,

$$P_k = A^TP_{k+1}A + Q - A^TP_{k+1}B(R+B^TP_{k+1}B)^{-1}B^TP_{k+1}A,$$

with boundary condition $P_h = Q_h$. As the following code illustrates, both 
the $P_k$s and $K_k$s converge as $k$ approaches $0$. This means that if $h$ 
is very large the optimal policy for $k$ small enough so that its values are 
still far from the horizon is approximately 

$$u_k = Kx_k$$ 

where

$$K = -(R+B^T PB)^{-1}B^TPA$$

and

$$P = A^TPA + Q - A^TPB(R+B^TPB)B^TPA.$$

This latter equation is the algebraic riccati equation. This equation can 
be shown to have a unique positive definite solution under mind assumption (e.g. 
if $Q$ is positive definite and if $(A,B)$ is controllable). A numerical method 
to obtain a solution to this equation is nothing more than starting with a finite 
horizon problem and letting the horizon tend to infinity, as considered here.

The example considers a double integrator: the state $x_k$ comprises position 
$y_k$ and velocity $v_k$, i.e., $x_k= [y_k \ \ v_k]^T$, and the control input 
$u_k$ is force. After discretizing the equation $\ddot{y}=u$ with a sampling 
period $\tau$ (zero order hold discretization) we obtain

$$A=e^{\left[ \matrix{ 0     & \tau \cr    0& 0 } \right]}={\left[ \matrix{1    & \tau \cr    0& 1 } \right]} , B = \int_0^\tau e^{As}ds\left[\matrix{0 \cr 1} \right] = \left[\matrix{\frac{\tau^2}{2} \cr \tau} \right].$$

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from course_functions import dlqr
%matplotlib ipympl

In [ ]:
def riccatiequations(A,B,Q,R,S,Qh,h):
    P = [ [] for i in range(h+1)]
    K = [ [] for i in range(h+1)]
    P[h] = Qh
    #Riccati equations
    for k in range(h-1,-1,-1):
        P[k] = A.T@P[k+1]@A + Q - (S+A.T@P[k+1]@B)*np.linalg.pinv(R+B.T@P[k+1]@B)@(S.T+B.T@P[k+1]@A)
        K[k] = -np.linalg.pinv(R+B.T@P[k+1]@B)@(S.T+B.T@P[k+1]@A)

    return K,P

In [ ]:
# define the system and problem matrices and discretize the process
Ac = np.array([[0 ,1],[0, 0]])
Bc = np.array([[0],[1]])
tau = 0.2
A, B, C = signal.cont2discrete((Ac, Bc, np.zeros((1,2)), np.array([0.2])), tau)[:3]
Q = np.array([[1, 0],[0, 1]])
S = np.array([[0],[0]])
R = 1
Qh = np.array([[1, 0],[0, 1]])
h = 100

#call the function that iterates the riccati equations and the dlqr function
K,P = riccatiequations(A,B,Q,R,S,Qh,h)
Kinf, Pinf, E = dlqr(A,B,Q,R)
Kinf = -Kinf

Compare with K{1}, P{1} with solution to Alg. Ric. eq.

Finite horizon gains K[0] and Ric. eq. sol. P[0]

In [ ]:
K[0]

In [ ]:
P[0]

Infinite horizon gains Kinf and Ric. eq. sol. Pinf

In [ ]:
Kinf

In [ ]:
Pinf

In [ ]:
p11 = np.zeros(h+1)
p21 = np.zeros(h+1)
p22 = np.zeros(h+1)
k1 = np.zeros(h)
k2 = np.zeros(h)
for i in range(h+1):
    p11[i] = P[i][0,0]
    p21[i] = P[i][0,1]
    p22[i] = P[i][1,1]
    if i!=h:
        k1[i] = K[i][0,0]
        k2[i] = K[i][0,1]


f = plt.figure()
ax = plt.gca()
kk_ = np.arange(0,h+1)
ax.plot(kk_, p11, label = "p1")
ax.plot(kk_, p21, label = "p2")
ax.plot(kk_, p22, label = "p3")
ax.set_xlabel("k")
ax.legend();

In [ ]:
f = plt.figure()
ax = plt.gca()
kk_ = np.arange(0,h)
ax.plot(kk_, k1, label = "k1")
ax.plot(kk_, k2, label = "k2")
ax.set_xlabel("k")
ax.legend();